### ACCRE is being slow right now so I'm just running this on workstation

In summary, yes, keeping the hash the same will keep the shuffle the same (and it does make a difference whether shuffle is true or not, and it does make a difference what the seed {not hash} is, and it makes a difference what the hash is).

All three sources of variability (Shuffle=False, different seed, different hash) came out differently, so there's no overlap.

**THE ONLY ASSUMPTION** that was made is that this transfers over to different CPUs that have the same architecture. That would have to be tested explicitly on ACCRE. However, we know that same hash => same output on different CPUs of the same architecture when the only thing governed by the hash is the initialization. Thus, it's safe to assume that the same hash results in the same results across all the random sources governed by the hash (and shuffle, as seen here, is one of those governed sources) when run on the same archetecture.

In [1]:
# Instance 1, Seed 0 and Shuffle=True

import sys
import numpy as np
import tensorflow as tf
import os
import random
sys.path.append('../../python_scripts/')
import correlations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, Dropout, GlobalAveragePooling2D,
                                    MaxPooling2D, Activation, Dense, Layer)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.callbacks import LearningRateScheduler, Callback
from tensorflow.keras import backend as K 

# Set seed value for python and np
seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
K.clear_session()
tf.random.set_seed(seed_value)


# Load predict datasets and whitened datasets
x_predict, y_predict = correlations.make_test_data()
x_train_new = np.load('../../data/cifar10_modified/x_train_new.npy')
y_train_new = np.load('../../data/cifar10_modified/y_train_new.npy')
x_test_new = np.load('../../data/cifar10_modified/x_test_new.npy')
y_test_new = np.load('../../data/cifar10_modified/y_test_new.npy')

# Function to make models
def init_model(architecture: str, seed: int):
    
    if (architecture == 'all_cnn_c'):
        model = Sequential()
        # model.add(Dropout(0.2, input_shape=x_train.shape[1:])) #input shape from keras cifar10 example
        model.add(Conv2D(96, (3, 3), input_shape=x_train_new.shape[1:], padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), 
                         bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), strides=2, padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), strides=2, padding='same',kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(10, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(GlobalAveragePooling2D())
        model.add(Activation('softmax'))
        
        model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9, clipnorm=500),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
    
#     elif (architecture =='vgg753'):
#         model = Sequential()
#         model.add(Conv2D(96, (7, 7), strides=2, input_shape=x_train.shape[1:], kernel_regularizer=l2(1e-5), 
#                          bias_regularizer=l2(1e-5), bias_initializer='zeros', kernel_initializer=he_normal(seed)))
#         model.add(BatchNormalization())
#         model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
#         model.add(Conv2D(128, (5, 5), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(MaxPooling2D(pool_size=(2, 2)))
#         model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
#         model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(GlobalAveragePooling2D())
#         model.add(Dense(10, activation='softmax'))

#         model.compile(optimizer=Adam(learning_rate=0.01, epsilon=0.1, clipnorm=500),
#                       loss='categorical_crossentropy',
#                       metrics=['accuracy'])

    else:
        raise

    return model

# Scheduler callback
def scheduler(epoch, lr):
    if (epoch == 200 or epoch == 250 or epoch == 300):
        return lr * 0.1
    return lr

lr_callback = LearningRateScheduler(scheduler)

# All_CNN_C
i = 0
print('Training All_CNN_C with seed', str(i))
all_cnn_c = init_model('all_cnn_c', seed=i)

instance_num = i
    
a_history = all_cnn_c.fit(
	x_train_new,
        y_train_new,
        batch_size=128,
        epochs=350,
        callbacks=[lr_callback],
        validation_data=(x_test_new, y_test_new),
        shuffle=True)
    


Making test data...
Done!
Training All_CNN_C with seed 0
Train on 50000 samples, validate on 10000 samples
Epoch 1/350
50000/50000 [==============================] - 185s 4ms/sample - loss: 2.3228 - accuracy: 0.1104 - val_loss: 2.3139 - val_accuracy: 0.1046
Epoch 2/350
50000/50000 [==============================] - 181s 4ms/sample - loss: 2.2968 - accuracy: 0.1388 - val_loss: 2.2927 - val_accuracy: 0.1318
Epoch 3/350
50000/50000 [==============================] - 181s 4ms/sample - loss: 2.2717 - accuracy: 0.1374 - val_loss: 2.2530 - val_accuracy: 0.1616
Epoch 4/350
50000/50000 [==============================] - 181s 4ms/sample - loss: 2.2389 - accuracy: 0.1629 - val_loss: 2.2521 - val_accuracy: 0.1606
Epoch 5/350
50000/50000 [==============================] - 180s 4ms/sample - loss: 2.1735 - accuracy: 0.2172 - val_loss: 2.1566 - val_accuracy: 0.2271
Epoch 6/350
 1536/50000 [..............................] - ETA: 2:47 - loss: 2.1303 - accuracy: 0.2429

KeyboardInterrupt: 

In [2]:
# Instance 2, Seed 0 and Shuffle=True
import sys
import numpy as np
import tensorflow as tf
import os
import random
sys.path.append('../../python_scripts/')
import correlations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, Dropout, GlobalAveragePooling2D,
                                    MaxPooling2D, Activation, Dense, Layer)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.callbacks import LearningRateScheduler, Callback
from tensorflow.keras import backend as K 

# Set seed value for python and np
seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
K.clear_session()
tf.random.set_seed(seed_value)


# Load predict datasets and whitened datasets
x_predict, y_predict = correlations.make_test_data()
x_train_new = np.load('../../data/cifar10_modified/x_train_new.npy')
y_train_new = np.load('../../data/cifar10_modified/y_train_new.npy')
x_test_new = np.load('../../data/cifar10_modified/x_test_new.npy')
y_test_new = np.load('../../data/cifar10_modified/y_test_new.npy')

# Function to make models
def init_model(architecture: str, seed: int):
    
    if (architecture == 'all_cnn_c'):
        model = Sequential()
        # model.add(Dropout(0.2, input_shape=x_train.shape[1:])) #input shape from keras cifar10 example
        model.add(Conv2D(96, (3, 3), input_shape=x_train_new.shape[1:], padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), 
                         bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), strides=2, padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), strides=2, padding='same',kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(10, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(GlobalAveragePooling2D())
        model.add(Activation('softmax'))
        
        model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9, clipnorm=500),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
    
#     elif (architecture =='vgg753'):
#         model = Sequential()
#         model.add(Conv2D(96, (7, 7), strides=2, input_shape=x_train.shape[1:], kernel_regularizer=l2(1e-5), 
#                          bias_regularizer=l2(1e-5), bias_initializer='zeros', kernel_initializer=he_normal(seed)))
#         model.add(BatchNormalization())
#         model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
#         model.add(Conv2D(128, (5, 5), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(MaxPooling2D(pool_size=(2, 2)))
#         model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
#         model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(GlobalAveragePooling2D())
#         model.add(Dense(10, activation='softmax'))

#         model.compile(optimizer=Adam(learning_rate=0.01, epsilon=0.1, clipnorm=500),
#                       loss='categorical_crossentropy',
#                       metrics=['accuracy'])

    else:
        raise

    return model

# Scheduler callback
def scheduler(epoch, lr):
    if (epoch == 200 or epoch == 250 or epoch == 300):
        return lr * 0.1
    return lr

lr_callback = LearningRateScheduler(scheduler)

# All_CNN_C
i = 0
print('Training All_CNN_C with seed', str(i))
all_cnn_c = init_model('all_cnn_c', seed=i)

instance_num = i
    
a_history = all_cnn_c.fit(
	x_train_new,
        y_train_new,
        batch_size=128,
        epochs=350,
        callbacks=[lr_callback],
        validation_data=(x_test_new, y_test_new),
        shuffle=True)
    


Making test data...
Done!
Training All_CNN_C with seed 0
Train on 50000 samples, validate on 10000 samples
Epoch 1/350
50000/50000 [==============================] - 181s 4ms/sample - loss: 2.3228 - accuracy: 0.1104 - val_loss: 2.3139 - val_accuracy: 0.1046
Epoch 2/350
50000/50000 [==============================] - 181s 4ms/sample - loss: 2.2968 - accuracy: 0.1388 - val_loss: 2.2927 - val_accuracy: 0.1318
Epoch 3/350
50000/50000 [==============================] - 181s 4ms/sample - loss: 2.2717 - accuracy: 0.1374 - val_loss: 2.2530 - val_accuracy: 0.1616
Epoch 4/350
 4736/50000 [=>............................] - ETA: 2:38 - loss: 2.2545 - accuracy: 0.1573

KeyboardInterrupt: 

In [3]:
# Instance 3, Seed 0 and Shuffle=True
import sys
import numpy as np
import tensorflow as tf
import os
import random
sys.path.append('../../python_scripts/')
import correlations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, Dropout, GlobalAveragePooling2D,
                                    MaxPooling2D, Activation, Dense, Layer)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.callbacks import LearningRateScheduler, Callback
from tensorflow.keras import backend as K 

# Set seed value for python and np
seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
K.clear_session()
tf.random.set_seed(seed_value)


# Load predict datasets and whitened datasets
x_predict, y_predict = correlations.make_test_data()
x_train_new = np.load('../../data/cifar10_modified/x_train_new.npy')
y_train_new = np.load('../../data/cifar10_modified/y_train_new.npy')
x_test_new = np.load('../../data/cifar10_modified/x_test_new.npy')
y_test_new = np.load('../../data/cifar10_modified/y_test_new.npy')

# Function to make models
def init_model(architecture: str, seed: int):
    
    if (architecture == 'all_cnn_c'):
        model = Sequential()
        # model.add(Dropout(0.2, input_shape=x_train.shape[1:])) #input shape from keras cifar10 example
        model.add(Conv2D(96, (3, 3), input_shape=x_train_new.shape[1:], padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), 
                         bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), strides=2, padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), strides=2, padding='same',kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(10, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(GlobalAveragePooling2D())
        model.add(Activation('softmax'))
        
        model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9, clipnorm=500),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
    
#     elif (architecture =='vgg753'):
#         model = Sequential()
#         model.add(Conv2D(96, (7, 7), strides=2, input_shape=x_train.shape[1:], kernel_regularizer=l2(1e-5), 
#                          bias_regularizer=l2(1e-5), bias_initializer='zeros', kernel_initializer=he_normal(seed)))
#         model.add(BatchNormalization())
#         model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
#         model.add(Conv2D(128, (5, 5), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(MaxPooling2D(pool_size=(2, 2)))
#         model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
#         model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(GlobalAveragePooling2D())
#         model.add(Dense(10, activation='softmax'))

#         model.compile(optimizer=Adam(learning_rate=0.01, epsilon=0.1, clipnorm=500),
#                       loss='categorical_crossentropy',
#                       metrics=['accuracy'])

    else:
        raise

    return model

# Scheduler callback
def scheduler(epoch, lr):
    if (epoch == 200 or epoch == 250 or epoch == 300):
        return lr * 0.1
    return lr

lr_callback = LearningRateScheduler(scheduler)

# All_CNN_C
i = 0
print('Training All_CNN_C with seed', str(i))
all_cnn_c = init_model('all_cnn_c', seed=i)

instance_num = i
    
a_history = all_cnn_c.fit(
	x_train_new,
        y_train_new,
        batch_size=128,
        epochs=350,
        callbacks=[lr_callback],
        validation_data=(x_test_new, y_test_new),
        shuffle=True)
    


Making test data...
Done!
Training All_CNN_C with seed 0
Train on 50000 samples, validate on 10000 samples
Epoch 1/350
50000/50000 [==============================] - 181s 4ms/sample - loss: 2.3228 - accuracy: 0.1104 - val_loss: 2.3139 - val_accuracy: 0.1046
Epoch 2/350
50000/50000 [==============================] - 180s 4ms/sample - loss: 2.2968 - accuracy: 0.1388 - val_loss: 2.2927 - val_accuracy: 0.1318
Epoch 3/350
50000/50000 [==============================] - 180s 4ms/sample - loss: 2.2717 - accuracy: 0.1374 - val_loss: 2.2530 - val_accuracy: 0.1616
Epoch 4/350
 1664/50000 [..............................] - ETA: 2:45 - loss: 2.2626 - accuracy: 0.1569

KeyboardInterrupt: 

In [5]:
# Instance 4, Seed 0 and Shuffle=False
import sys
import numpy as np
import tensorflow as tf
import os
import random
sys.path.append('../../python_scripts/')
import correlations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, Dropout, GlobalAveragePooling2D,
                                    MaxPooling2D, Activation, Dense, Layer)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.callbacks import LearningRateScheduler, Callback
from tensorflow.keras import backend as K 

# Set seed value for python and np
seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
K.clear_session()
tf.random.set_seed(seed_value)


# Load predict datasets and whitened datasets
x_predict, y_predict = correlations.make_test_data()
x_train_new = np.load('../../data/cifar10_modified/x_train_new.npy')
y_train_new = np.load('../../data/cifar10_modified/y_train_new.npy')
x_test_new = np.load('../../data/cifar10_modified/x_test_new.npy')
y_test_new = np.load('../../data/cifar10_modified/y_test_new.npy')

# Function to make models
def init_model(architecture: str, seed: int):
    
    if (architecture == 'all_cnn_c'):
        model = Sequential()
        # model.add(Dropout(0.2, input_shape=x_train.shape[1:])) #input shape from keras cifar10 example
        model.add(Conv2D(96, (3, 3), input_shape=x_train_new.shape[1:], padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), 
                         bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), strides=2, padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), strides=2, padding='same',kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(10, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(GlobalAveragePooling2D())
        model.add(Activation('softmax'))
        
        model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9, clipnorm=500),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
    
#     elif (architecture =='vgg753'):
#         model = Sequential()
#         model.add(Conv2D(96, (7, 7), strides=2, input_shape=x_train.shape[1:], kernel_regularizer=l2(1e-5), 
#                          bias_regularizer=l2(1e-5), bias_initializer='zeros', kernel_initializer=he_normal(seed)))
#         model.add(BatchNormalization())
#         model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
#         model.add(Conv2D(128, (5, 5), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(MaxPooling2D(pool_size=(2, 2)))
#         model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
#         model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(GlobalAveragePooling2D())
#         model.add(Dense(10, activation='softmax'))

#         model.compile(optimizer=Adam(learning_rate=0.01, epsilon=0.1, clipnorm=500),
#                       loss='categorical_crossentropy',
#                       metrics=['accuracy'])

    else:
        raise

    return model

# Scheduler callback
def scheduler(epoch, lr):
    if (epoch == 200 or epoch == 250 or epoch == 300):
        return lr * 0.1
    return lr

lr_callback = LearningRateScheduler(scheduler)

# All_CNN_C
i = 0
print('Training All_CNN_C with seed', str(i))
all_cnn_c = init_model('all_cnn_c', seed=i)

instance_num = i
    
a_history = all_cnn_c.fit(
	x_train_new,
        y_train_new,
        batch_size=128,
        epochs=350,
        callbacks=[lr_callback],
        validation_data=(x_test_new, y_test_new),
        shuffle=False)
    


Making test data...
Done!
Training All_CNN_C with seed 0
Train on 50000 samples, validate on 10000 samples
Epoch 1/350
50000/50000 [==============================] - 181s 4ms/sample - loss: 2.3237 - accuracy: 0.0957 - val_loss: 2.3091 - val_accuracy: 0.1032
Epoch 2/350
50000/50000 [==============================] - 181s 4ms/sample - loss: 2.2932 - accuracy: 0.1317 - val_loss: 2.2963 - val_accuracy: 0.0981
Epoch 3/350
20992/50000 [===========>..................] - ETA: 1:41 - loss: 2.2938 - accuracy: 0.1016

KeyboardInterrupt: 

In [7]:
# Instance 5, Seed 100 and Shuffle=True
import sys
import numpy as np
import tensorflow as tf
import os
import random
sys.path.append('../../python_scripts/')
import correlations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, Dropout, GlobalAveragePooling2D,
                                    MaxPooling2D, Activation, Dense, Layer)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.callbacks import LearningRateScheduler, Callback
from tensorflow.keras import backend as K 

# Set seed value for python and np
seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
K.clear_session()
tf.random.set_seed(seed_value)


# Load predict datasets and whitened datasets
x_predict, y_predict = correlations.make_test_data()
x_train_new = np.load('../../data/cifar10_modified/x_train_new.npy')
y_train_new = np.load('../../data/cifar10_modified/y_train_new.npy')
x_test_new = np.load('../../data/cifar10_modified/x_test_new.npy')
y_test_new = np.load('../../data/cifar10_modified/y_test_new.npy')

# Function to make models
def init_model(architecture: str, seed: int):
    
    if (architecture == 'all_cnn_c'):
        model = Sequential()
        # model.add(Dropout(0.2, input_shape=x_train.shape[1:])) #input shape from keras cifar10 example
        model.add(Conv2D(96, (3, 3), input_shape=x_train_new.shape[1:], padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), 
                         bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), strides=2, padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), strides=2, padding='same',kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(10, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(GlobalAveragePooling2D())
        model.add(Activation('softmax'))
        
        model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9, clipnorm=500),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
    
#     elif (architecture =='vgg753'):
#         model = Sequential()
#         model.add(Conv2D(96, (7, 7), strides=2, input_shape=x_train.shape[1:], kernel_regularizer=l2(1e-5), 
#                          bias_regularizer=l2(1e-5), bias_initializer='zeros', kernel_initializer=he_normal(seed)))
#         model.add(BatchNormalization())
#         model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
#         model.add(Conv2D(128, (5, 5), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(MaxPooling2D(pool_size=(2, 2)))
#         model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
#         model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(GlobalAveragePooling2D())
#         model.add(Dense(10, activation='softmax'))

#         model.compile(optimizer=Adam(learning_rate=0.01, epsilon=0.1, clipnorm=500),
#                       loss='categorical_crossentropy',
#                       metrics=['accuracy'])

    else:
        raise

    return model

# Scheduler callback
def scheduler(epoch, lr):
    if (epoch == 200 or epoch == 250 or epoch == 300):
        return lr * 0.1
    return lr

lr_callback = LearningRateScheduler(scheduler)

# All_CNN_C
i = 100
print('Training All_CNN_C with seed', str(i))
all_cnn_c = init_model('all_cnn_c', seed=i)

instance_num = i
    
a_history = all_cnn_c.fit(
	x_train_new,
        y_train_new,
        batch_size=128,
        epochs=350,
        callbacks=[lr_callback],
        validation_data=(x_test_new, y_test_new),
        shuffle=True)
    


Making test data...
Done!
Training All_CNN_C with seed 100
Train on 50000 samples, validate on 10000 samples
Epoch 1/350
50000/50000 [==============================] - 181s 4ms/sample - loss: 2.2570 - accuracy: 0.1619 - val_loss: 2.1842 - val_accuracy: 0.2111
Epoch 2/350
 4608/50000 [=>............................] - ETA: 2:38 - loss: 2.1913 - accuracy: 0.2020

KeyboardInterrupt: 

In [8]:
# Instance 4, Seed 0 Shuffle True HASH 100
import sys
import numpy as np
import tensorflow as tf
import os
import random
sys.path.append('../../python_scripts/')
import correlations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, Dropout, GlobalAveragePooling2D,
                                    MaxPooling2D, Activation, Dense, Layer)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.callbacks import LearningRateScheduler, Callback
from tensorflow.keras import backend as K 

# Set seed value for python and np
seed_value= 100
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
K.clear_session()
tf.random.set_seed(seed_value)


# Load predict datasets and whitened datasets
x_predict, y_predict = correlations.make_test_data()
x_train_new = np.load('../../data/cifar10_modified/x_train_new.npy')
y_train_new = np.load('../../data/cifar10_modified/y_train_new.npy')
x_test_new = np.load('../../data/cifar10_modified/x_test_new.npy')
y_test_new = np.load('../../data/cifar10_modified/y_test_new.npy')

# Function to make models
def init_model(architecture: str, seed: int):
    
    if (architecture == 'all_cnn_c'):
        model = Sequential()
        # model.add(Dropout(0.2, input_shape=x_train.shape[1:])) #input shape from keras cifar10 example
        model.add(Conv2D(96, (3, 3), input_shape=x_train_new.shape[1:], padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), 
                         bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), strides=2, padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), strides=2, padding='same',kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(10, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(GlobalAveragePooling2D())
        model.add(Activation('softmax'))
        
        model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9, clipnorm=500),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
    
#     elif (architecture =='vgg753'):
#         model = Sequential()
#         model.add(Conv2D(96, (7, 7), strides=2, input_shape=x_train.shape[1:], kernel_regularizer=l2(1e-5), 
#                          bias_regularizer=l2(1e-5), bias_initializer='zeros', kernel_initializer=he_normal(seed)))
#         model.add(BatchNormalization())
#         model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
#         model.add(Conv2D(128, (5, 5), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(MaxPooling2D(pool_size=(2, 2)))
#         model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
#         model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(GlobalAveragePooling2D())
#         model.add(Dense(10, activation='softmax'))

#         model.compile(optimizer=Adam(learning_rate=0.01, epsilon=0.1, clipnorm=500),
#                       loss='categorical_crossentropy',
#                       metrics=['accuracy'])

    else:
        raise

    return model

# Scheduler callback
def scheduler(epoch, lr):
    if (epoch == 200 or epoch == 250 or epoch == 300):
        return lr * 0.1
    return lr

lr_callback = LearningRateScheduler(scheduler)

# All_CNN_C
i = 0
print('Training All_CNN_C with seed', str(i))
all_cnn_c = init_model('all_cnn_c', seed=i)

instance_num = i
    
a_history = all_cnn_c.fit(
	x_train_new,
        y_train_new,
        batch_size=128,
        epochs=350,
        callbacks=[lr_callback],
        validation_data=(x_test_new, y_test_new),
        shuffle=True)
    


Making test data...
Done!
Training All_CNN_C with seed 0
Train on 50000 samples, validate on 10000 samples
Epoch 1/350
50000/50000 [==============================] - 181s 4ms/sample - loss: 2.3231 - accuracy: 0.1043 - val_loss: 2.3190 - val_accuracy: 0.1196
Epoch 2/350
 3584/50000 [=>............................] - ETA: 2:41 - loss: 2.3126 - accuracy: 0.1073

KeyboardInterrupt: 